In [1]:
import gurobipy   as     gp
from   gurobipy   import GRB
from drawpointFunction  import drawpointFunction
from globalFunctions import getNumberOfBlocksInADimension
import pandas as pd

In [2]:
class UndergroundModel:
    #Underground Model + Crown Pillar Restrictions.
    def __init__(self, database, numberOfPeriods):
        self.database = database
        self.numberOfPeriods = numberOfPeriods
        self.DP_init = 0       #### Tipo de extracción
        self.desc = 0.1
        self.colHeight = 300
        self.pos_x = 430     
        self.pos_y = 550     
        self.pos_z = 780
        self.pos_x_f = 730     
        self.pos_y_f = 910     
        self.p_t = 3791.912
        self.epsilon = 1
        self.orientationToExtractTheDrawpoints = 0

    def execute(self):
        self.getUndergroundVariablesFromCSV()
        self.getUndergroundInfo()
        self.setUndergroundParameters()
        self.setUndergroundMineLimits()
        self.setUndergroundVariables()
        self.setModelandGetResults()
        return self.objValue, self.variableValues, self.runtime, self.gap
    
    def getUndergroundVariablesFromCSV(self):
        self.undergroundBlocksLength = self.database['X'].to_dict()             
        self.undergroundBlocksWidth  = self.database['Y'].to_dict()             
        self.undergroundBlocksHeight = self.database['Z'].to_dict()             
        self.undergroundBlockTonnage = self.database['Ton'].to_dict()              
        self.undergroundBlockMineral  = self.database['Mineral'].to_dict()          
        self.undergroundBlockRecovery  = self.database['Recuperación'].to_dict()     
        self.undergroundCopperLaw  = self.database['%Cu'].to_dict()
        self.undergroundExtractionFixedCosts = self.database['CPlanta CA'].to_dict()
        self.undergroundVariableExtractionCosts = self.database['CMina CA'].to_dict()
        self.undergroundCP_S = self.database['CPlanta S'].to_dict()
        self.undergroundCM_S = self.database['CMINA S'].to_dict() 
    
    def getUndergroundInfo(self):
        self.undergroundBlocks = [i for i in range(len(self.undergroundBlocksLength.values()))]

    def setUndergroundParameters(self):
        #Underground Parameters
        self.t_S   = {period : period + 1 for period in range(self.numberOfPeriods)}
        self.MU_mt = {period : 25806600.0  for period in range(self.numberOfPeriods)} #Tonleage es mina
        self.ML_mt = {period : 0.0  for period in range(self.numberOfPeriods)}
        self.MU_pt = {period : 17777880.0   for period in range(self.numberOfPeriods)}#Mineral es planta
        self.ML_pt = {period : 0.0 for period in range(self.numberOfPeriods)}
        self.qU_dt = {period : 1 for period in range(self.numberOfPeriods)}
        self.qL_dt = {period : 0 for period in range(self.numberOfPeriods)}
        self.A_d   = {period : 2 for period in range(self.numberOfPeriods)}
        self.NU_nt = {period : 59 for period in range(self.numberOfPeriods)} 
        self.NL_nt = {period : 0 for period in range(self.numberOfPeriods)}
        self.N_t   = {period : 57 * (1 + period) for period in range(self.numberOfPeriods)}
        self.RL_dt = {period : 0.3 for period in range(self.numberOfPeriods)}
        self.RU_dt = {period : 0.7 for period in range(self.numberOfPeriods)}

    def setUndergroundVariables(self):
        self.drawpoint, self.G_d, self.Q_d,self.LEY_D, self.C_pdt, self.C_mdt, self.predecessor, self.x_draw,self.y_draw, self.z_draw = drawpointFunction(
                        self.pos_x, self.pos_y, self.pos_z, self.colHeight, self.DP_init, self.undergroundBlocksLengthLimits, self.undergroundBlocksWidthLimits, self.undergroundBlocksHeightLimits, self.undergroundBlockTonnage, self.undergroundCP_S, self.undergroundCM_S, self.undergroundBlockMineral,
                        self.undergroundCopperLaw, self.pos_x_f, self.pos_y_f,self.orientationToExtractTheDrawpoints)
        #G_d = waste and minneral tonnelage 
        #Q_d = minneral tonnelage
    
    def setUndergroundMineLimits(self):
        self.undergroundBlocksLengthLimits = getNumberOfBlocksInADimension(self.undergroundBlocksLength)
        self.undergroundBlocksWidthLimits = getNumberOfBlocksInADimension(self.undergroundBlocksWidth)
        self.undergroundBlocksHeightLimits = getNumberOfBlocksInADimension(self.undergroundBlocksHeight)

    def setModelandGetResults(self):
        self.objValue, self.variableValues, self.runtime, self.gap = self.setUndergroundModel()#v_p, theta_opt, w_opt)

    def setUndergroundModel(self):#,v_p, theta_opt, w_opt):
        self.undergroundModel = gp.Model(name = 'Modelo Integrado')
        self.undergroundModel.Params.TimeLimit = 3600

        #14. Naturaleza de las variables
        x_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="x")
        y_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.CONTINUOUS, name="y")
        z_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="z")

        #1. Restricción sobre la cantidad de tonelaje máxima y mínima a extraer en cada periodo.
        Ton_Up = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.G_d[d] for d in self.drawpoint) <= self.MU_mt[ti] for ti in self.t_S),
                                         "Min_max")
        
        Ton_low = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.G_d[d] for d in self.drawpoint) >= self.ML_mt[ti] for ti in self.t_S),
                                            "Min_min")
        #2. Restricción sobre la cantidad de material máxima y mínima a procesar en cada periodo.
        Mat_Up = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]* self.Q_d[d] for d in self.drawpoint) <= self.MU_pt[ti] for ti in self.t_S),
                                            "Mat_max")

        Mat_low = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]* self.Q_d[d] for d in self.drawpoint) >= self.ML_pt[ti] for ti in self.t_S)
                                            , "Mat_min")
        #3. Rango de leyes máximas y mínimas a procesar
        GQC_Up = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.LEY_D[d]*self.G_d[d] for d in self.drawpoint) <=
                                self.qU_dt[ti] * gp.quicksum(self.Q_d[d] * y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_Up")

        GQC_low = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.LEY_D[d]*self.G_d[d] for d in self.drawpoint) >=
                                self.qL_dt[ti] * gp.quicksum(self.Q_d[d] * y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_low")

        #4. Todos los puntos de extracción deben ser iniciados en el largo de la extracción
        Drawp_init = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint), "Drawp_init")

        #5. Los puntos de extracción deben ser activados al menos en el mismo periodo para que se inicie la extracción 
        Drawpextract_61 = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, tau] for tau in range(ti+1)) >= z_dt[d, ti]  
                                            for d in self.drawpoint for ti in self.t_S), "Drawpextract_61")


        #6. Existe una cantidad máxima y mínima de drawpoints a abrir en cada periodo.
        Drawpextract_64_1 = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) <= self.NU_nt[ti] for ti 
                                                        in self.t_S)
                                                        ,"Drawpextract_64_1")

        Drawpextract_64_2 = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) >= self.NL_nt[ti] for ti 
                                                        in self.t_S)
                                                        , "Drawpextract_64_2")

        #7. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        Drawpextract_65 = self.undergroundModel.addConstrs((gp.quicksum(z_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")


        #8. Si iniciamos la extracción de un drawpoint esta debe durar por su duraci ́on determinada.
        ## Un drawpoint solamente puede ser extraido por un preiodo pre determinado (A_d)
        Drawpextract_62 = self.undergroundModel.addConstrs((gp.quicksum(z_dt[d, ti] for ti in self.t_S)  <= self.A_d[ti]  for d in self.drawpoint
                                                    for ti in self.t_S), "Drawp_62")

        ## Una vez se inicia extrayendo de un drawpoint, se continua extrayendo sin interrupción
        Drawpextract_63 = self.undergroundModel.addConstrs((self.A_d[ti] *(z_dt[d, ti] - z_dt[d, ti+1]) 
                                            - gp.quicksum(z_dt[d, tau] for tau in range(ti+1)) <= 0 
                                            for d in self.drawpoint for ti in range(0,max(self.t_S))), "Drawpextract_63")

        #9. Relación de variables, el porcentaje a extraer es 0 si no se extra un drawpoint.
        Drawpextract_66 = self.undergroundModel.addConstrs((y_dt[d, ti] <= z_dt[d, ti] for d in self.drawpoint for ti in self.t_S),
                                                    "Drawpextract_66")

        #10. Existe una tasa m ́ınima de extracci ́on para cada drawpoint a extraer.
        Drawpextract_67_1 = self.undergroundModel.addConstrs((self.RL_dt[ti] * z_dt[d, ti]  <=  y_dt[d, ti] for d in self.drawpoint
                                                        for ti in self.t_S), "Drawpextract_67_1")

        #11. La altura a extraer debe ser mayor a una cantidad m ́ınima.
        #rest_11 = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d,ti] for ti in self.t_S)>= self.colHeight for d in self.drawpoint))

        #12. No podemos extraer más del 100 % de un drawpoint.
        Reserver_cnst = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint),
                                                    "Reserver_cnst")

        #13. Si se activa un drawpoint, se extrae en ese periodo
        rest_13 = self.undergroundModel.addConstrs(x_dt[d,ti] <= z_dt[d, ti] for d in self.drawpoint for ti in self.t_S)

        #14. Naturaleza de variables.

        #15. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        rest_15= self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")

        

        #16. Restricción sobre el inicio de la extracci ́on de los drawpoints.
        DP_Sup = self.undergroundModel.addConstrs((gp.quicksum(x_dt[self.predecessor[l][0], s]*(max(self.t_S)-s+1) for s in self.t_S) <=
                                    gp.quicksum(x_dt[self.predecessor[l][1], s]*(max(self.t_S)-s+1) for s in self.t_S)  
                                    for l in range(len(self.predecessor))), "DP_Sup")

        #17. Restricci ́on sobre la extracci ́on de los drawpoints.
        restricion_z_dt = self.undergroundModel.addConstrs((gp.quicksum(z_dt[self.predecessor[l][0], s]*(max(self.t_S)-s+1) for s in self.t_S) <=
                                    gp.quicksum(z_dt[self.predecessor[l][1], s]*(max(self.t_S)-s+1) for s in self.t_S)  
                                    for l in range(len(self.predecessor))), "DP_Sup")



        
        #Función objetivo
        undergroundObjectiveFunction = gp.quicksum(y_dt[d, ti]*((((self.p_t * self.LEY_D[d] -self.C_pdt[d] ) * self.Q_d[d])-(self.C_mdt[d]*self.G_d[d]))/
                                        ((1+self.desc)**(self.t_S[ti]))) for ti in self.t_S for d in self.drawpoint)
        
        """
        #FALTA DEFINIR LOS CONJUNTOS D_v, V

        
        #Restricciones del crown pillar

        #Variable 1 si y solo si el crown pillar esta ubicado en la elevaci ́on v, 0 en otro caso.
        w_v = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="w")

        #Restricciones del crown pillar
        pillar_2 = self.undergroundModel.addConstrs(gp.quicksum(x_dt[d, ti] for d in D_v)<=1- w_v[v] for v in V for ti in self.t_S)

        pillar_3 = self.undergroundModel.addConstrs(gp.quicksum(w_v[v])==1 for v in V)

        #Restricción extra para iterar
        if v_p < theta_opt - self.epsilon:
            optimal_cut = self.undergroundModel.addConstrs(theta <= v_p + gp.quicksum(mu_v*(w_v[v]-w_opt[v])for v in V))
        """
       
        self.undergroundModel.setObjective(undergroundObjectiveFunction, GRB.MAXIMIZE)
        self.undergroundModel.Params.MIPGap = 0.01
        
        self.undergroundModel.optimize()
        lista_variable_Integrado = (self.undergroundModel.getAttr(GRB.Attr.X, self.undergroundModel.getVars()))
        solucion = self.undergroundModel.objVal
        runtime = self.undergroundModel.Runtime
        gap_f = self.undergroundModel.MIPGap
        #print(y_dt)
        return solucion, lista_variable_Integrado, runtime, gap_f

In [4]:
undergroundMineDataframe = pd.read_excel("C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/integratedModel/Modelo_F_OG.xlsx", engine="openpyxl")
#undergroundMineDataframe =  pd.read_excel("C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/Database/integratedModel/Modelo_F_OG.xlsx", engine="openpyxl")

In [5]:
numberOfPeriods = 5

In [6]:
self.undergroundModel = self.undergroundModel(undergroundMineDataframe,numberOfPeriods)

NameError: name 'self' is not defined

In [9]:
self.undergroundModel.LEY_D

[0.009328944444444445,
 0.010589311111111112,
 0.01033133888888889,
 0.010599466666666665,
 0.010581772222222221,
 0.010654027777777778,
 0.011104905555555555,
 0.01136338888888889,
 0.010552227777777779,
 0.010567166666666667,
 0.010454288888888889,
 0.010486294444444447,
 0.0100215,
 0.009380422222222221,
 0.006503388888888889,
 0.0099787,
 0.012452522222222219,
 0.013366816666666666,
 0.013363166666666666,
 0.013864777777777776,
 0.013546077777777776,
 0.013270788888888888,
 0.013091827777777778,
 0.01344955,
 0.013178027777777778,
 0.012987261111111113,
 0.013325927777777776,
 0.012640361111111111,
 0.01162978888888889,
 0.008956866666666665,
 0.010591044444444444,
 0.012245772222222222,
 0.013349727777777773,
 0.014056305555555555,
 0.013906966666666666,
 0.014450666666666669,
 0.014772377777777778,
 0.014228044444444447,
 0.014290833333333334,
 0.01396365,
 0.014453411111111113,
 0.013388855555555556,
 0.013506394444444443,
 0.01219411111111111,
 0.009114844444444447,
 0.01120103

In [7]:
undergroundObjValue, undergroundVariableValues, undergroundRuntime, undergroundGap = self.undergroundModel.execute()

Si funciono
Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-30
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 5988 rows, 2700 columns and 30400 nonzeros
Model fingerprint: 0x0b63baca
Variable types: 900 continuous, 1800 integer (1800 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+05]
  Objective range  [6e+05, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+07]
Presolve removed 2246 rows and 296 columns
Presolve time: 0.12s
Presolved: 3742 rows, 2404 columns, 17673 nonzeros
Variable types: 900 continuous, 1504 integer (1504 binary)

Root relaxation: infeasible, 753 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

GurobiError: Unable to retrieve attribute 'X'

In [13]:
blockHeight, minHeight, maxHeight, numOfDifferentsBlocks = self.undergroundModel.undergroundBlocksHeightLimits

In [14]:
from itertools import chain

V = [height for height in chain(range(maxHeight,minHeight,blockHeight), [minHeight])]


In [15]:
V

[530,
 540,
 550,
 560,
 570,
 580,
 590,
 600,
 610,
 620,
 630,
 640,
 650,
 660,
 670,
 680,
 690,
 700,
 710,
 720,
 730,
 740,
 750,
 760,
 770,
 780,
 790,
 800,
 810,
 820,
 830,
 840,
 850,
 860,
 870,
 880,
 890,
 900,
 910,
 920,
 930,
 940,
 950,
 960,
 970,
 980,
 990,
 1000,
 1010,
 1020,
 1030,
 1040,
 1050,
 1060,
 1070,
 1080,
 1090,
 1100,
 1110,
 1120,
 1130,
 1140,
 1150,
 1160,
 1170,
 1180,
 1190,
 1200,
 1210,
 1220,
 1230,
 1240,
 1250,
 1260,
 1270,
 1280,
 1290,
 1300,
 1310,
 1320]

In [31]:
V

[530,
 540,
 550,
 560,
 570,
 580,
 590,
 600,
 610,
 620,
 630,
 640,
 650,
 660,
 670,
 680,
 690,
 700,
 710,
 720,
 730,
 740,
 750,
 760,
 770,
 780,
 790,
 800,
 810,
 820,
 830,
 840,
 850,
 860,
 870,
 880,
 890,
 900,
 910,
 920,
 930,
 940,
 950,
 960,
 970,
 980,
 990,
 1000,
 1010,
 1020,
 1030,
 1040,
 1050,
 1060,
 1070,
 1080,
 1090,
 1100,
 1110,
 1120,
 1130,
 1140,
 1150,
 1160,
 1170,
 1180,
 1190,
 1200,
 1210,
 1220,
 1230,
 1240,
 1250,
 1260,
 1270,
 1280,
 1290,
 1300,
 1310,
 1320]

In [66]:
rho_v = [1 - (v - maxHeight)/(minHeight -maxHeight) for v in V]
rho_v

[1.0,
 0.9873417721518988,
 0.9746835443037974,
 0.9620253164556962,
 0.9493670886075949,
 0.9367088607594937,
 0.9240506329113924,
 0.9113924050632911,
 0.8987341772151899,
 0.8860759493670887,
 0.8734177215189873,
 0.8607594936708861,
 0.8481012658227848,
 0.8354430379746836,
 0.8227848101265822,
 0.810126582278481,
 0.7974683544303798,
 0.7848101265822784,
 0.7721518987341772,
 0.759493670886076,
 0.7468354430379747,
 0.7341772151898734,
 0.7215189873417722,
 0.7088607594936709,
 0.6962025316455696,
 0.6835443037974683,
 0.6708860759493671,
 0.6582278481012658,
 0.6455696202531646,
 0.6329113924050633,
 0.620253164556962,
 0.6075949367088608,
 0.5949367088607596,
 0.5822784810126582,
 0.5696202531645569,
 0.5569620253164558,
 0.5443037974683544,
 0.5316455696202531,
 0.5189873417721519,
 0.5063291139240507,
 0.49367088607594933,
 0.4810126582278481,
 0.4683544303797469,
 0.45569620253164556,
 0.44303797468354433,
 0.430379746835443,
 0.4177215189873418,
 0.40506329113924056,
 0.3924

In [67]:
rho_v

[1.0,
 0.9873417721518988,
 0.9746835443037974,
 0.9620253164556962,
 0.9493670886075949,
 0.9367088607594937,
 0.9240506329113924,
 0.9113924050632911,
 0.8987341772151899,
 0.8860759493670887,
 0.8734177215189873,
 0.8607594936708861,
 0.8481012658227848,
 0.8354430379746836,
 0.8227848101265822,
 0.810126582278481,
 0.7974683544303798,
 0.7848101265822784,
 0.7721518987341772,
 0.759493670886076,
 0.7468354430379747,
 0.7341772151898734,
 0.7215189873417722,
 0.7088607594936709,
 0.6962025316455696,
 0.6835443037974683,
 0.6708860759493671,
 0.6582278481012658,
 0.6455696202531646,
 0.6329113924050633,
 0.620253164556962,
 0.6075949367088608,
 0.5949367088607596,
 0.5822784810126582,
 0.5696202531645569,
 0.5569620253164558,
 0.5443037974683544,
 0.5316455696202531,
 0.5189873417721519,
 0.5063291139240507,
 0.49367088607594933,
 0.4810126582278481,
 0.4683544303797469,
 0.45569620253164556,
 0.44303797468354433,
 0.430379746835443,
 0.4177215189873418,
 0.40506329113924056,
 0.3924